In [2]:
pip install pyyaml


You should consider upgrading via the '/home/vahid/.virtualenvs/py37-tf1-gpu/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [39]:
import yaml
import numpy as np
import pandas as pd

In [61]:
with open('/home/vahid/Desktop/python-mobo-wrong-2.yml') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
data.keys()

dict_keys(['name', 'version', 'autoConfigApproach', 'iterations', 'workload', 'allSettings'])

In [62]:
print('name', data['name'])
print('strategy', data['autoConfigApproach'])
print('number iterations', len(data['iterations']))
print('workload', data['workload'])

name bookstore-mobo
strategy PythonRunner
number iterations 25
workload {'args_duration': '90s', 'args_editbook_prob': '0.6', 'args_getbook_prob': '0.1', 'args_login_prob': '0.3', 'args_sleep_duration': '0.5', 'args_vus': '125'}


In [63]:
sla_value = 0.250
temp = {'cpu':[],'iteration':[],'meets': []}
for iter_id, iteration in enumerate(data['iterations']):
    temp['iteration'].append(iter_id)
    
    config = iteration['configurations']
    print(config)
    cpu = 0
    for _, c in config.items():
        cpu += c['cpu'] * c['replicacount']
    temp['cpu'].append(cpu/1000)
    
    aggData = iteration['aggregatedData']
    
    meets = True
    for _, values in aggData['responseTimes'].items():
#         print(np.percentile(values, 95))
        print(len(values))
        if (np.percentile(values, 95) > sla_value):
            meets = False
    print('-----')
    temp['meets'].append(meets)
#     break
df = pd.DataFrame(temp)
df.head(40)

{'auth': {'resourcetype': 'Deployment', 'replicacount': 1, 'cpu': 500, 'memory': 256, 'environmentvalues': {}}, 'books': {'resourcetype': 'Deployment', 'replicacount': 1, 'cpu': 500, 'memory': 256, 'environmentvalues': {}}, 'gateway': {'resourcetype': 'Deployment', 'replicacount': 1, 'cpu': 500, 'memory': 256, 'environmentvalues': {}}}
3473
584
1687
-----
{'auth': {'resourcetype': 'Deployment', 'replicacount': 1, 'cpu': 727, 'memory': 256, 'environmentvalues': {}}, 'books': {'resourcetype': 'Deployment', 'replicacount': 6, 'cpu': 956, 'memory': 256, 'environmentvalues': {}}, 'gateway': {'resourcetype': 'Deployment', 'replicacount': 7, 'cpu': 980, 'memory': 256, 'environmentvalues': {}}}
3371
561
1637
-----
{'auth': {'resourcetype': 'Deployment', 'replicacount': 3, 'cpu': 999, 'memory': 256, 'environmentvalues': {}}, 'books': {'resourcetype': 'Deployment', 'replicacount': 3, 'cpu': 908, 'memory': 256, 'environmentvalues': {}}, 'gateway': {'resourcetype': 'Deployment', 'replicacount': 6,

,cpu,iteration,meets
0,1.500,0,False
1,13.323,1,False
2,11.517,2,False
3,10.573,3,False
4,13.842,4,False
5,14.633,5,False
6,3.210,6,False
7,7.308,7,False
8,3.808,8,False
9,19.400,9,True


In [30]:
data['iterations'][0]['configurations']

{'auth': {'resourcetype': 'Deployment',
  'replicacount': 3,
  'cpu': 999,
  'memory': 256,
  'environmentvalues': {}},
 'books': {'resourcetype': 'Deployment',
  'replicacount': 3,
  'cpu': 908,
  'memory': 256,
  'environmentvalues': {}},
 'gateway': {'resourcetype': 'Deployment',
  'replicacount': 6,
  'cpu': 966,
  'memory': 256,
  'environmentvalues': {}}}

In [31]:
data['iterations'][1]['configurations']

{'auth': {'resourcetype': 'Deployment',
  'replicacount': 3,
  'cpu': 999,
  'memory': 256,
  'environmentvalues': {}},
 'books': {'resourcetype': 'Deployment',
  'replicacount': 3,
  'cpu': 908,
  'memory': 256,
  'environmentvalues': {}},
 'gateway': {'resourcetype': 'Deployment',
  'replicacount': 6,
  'cpu': 966,
  'memory': 256,
  'environmentvalues': {}}}

In [60]:
# split CPU cores, based on share of each service, create multiple containers. at most 1 CPU core per container

# https://github.cpom/ppgaluzio/MOBOpt
# @article{GALUZIO2020100520,
# title = "MOBOpt — multi-objective Bayesian optimization",
# journal = "SoftwareX",
# volume = "12",
# pages = "100520",
# year = "2020",
# issn = "2352-7110",
# doi = "https://doi.org/10.1016/j.softx.2020.100520",
# url = "http://www.sciencedirect.com/science/article/pii/S2352711020300911",
# author = "Paulo Paneque Galuzio and Emerson Hochsteiner [de Vasconcelos Segundo] and Leandro dos Santos Coelho and Viviana Cocco Mariani"
# }
import json
import mobopt as mo
import numpy as np
import sys
core_count = 20000 # make sure this is the same as what you have in config file of acfg as ConfigurationValidation.TotalCPU
cache = {}
global counter
counter = 0
import time
def objective(x):
    global counter
    print("A", counter)
    counter += 1
    
    return np.array([1,2,1,1])
PB = np.asarray([
    [0.06, 0.94],
    [0.06, 0.94],
    [0.06, 0.94],
    [0.06, 0.94]
])
NParam = PB.shape[0]

Optimizer = mo.MOBayesianOpt(target=objective,
                             NObj=4,
                             pbounds=PB,
                             verbose=False,
                             max_or_min='min', # whether the optimization problem is a maximization problem ('max'), or a minimization one ('min')
                             RandomSeed=10)
Optimizer.initialize(init_points=5) 
# there is no minimize function. maximize() starts optimization. Performs minimizing or maximizing based on max_or_min
front, pop = Optimizer.maximize(n_iter=100,
                                prob=0.1,
                                ReduceProb=False,
                                q=0.5)
print('done')

A 0
A 1
A 2
A 3
A 4


/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/v

A 5


/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/v

A 6
A 7


...

A 102


/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/v

A 103


/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/vahid/.virtualenvs/py37-tf1-gpu/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/home/v

A 104
done


In [56]:
!pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.1-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
You should consider upgrading via the '/home/vahid/.virtualenvs/py37-tf1-gpu/bin/python -m pip install --upgrade pip' command.
